In [1]:
%matplotlib widget

import sys  
sys.path.insert(0, '/home/cbisot/pycode/MscThesis/')
import pandas as pd
from sample.util import get_dates_datetime, get_dirname, get_plate_number, get_postion_number

import ast
from sample.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
from sample.pipeline.functions.node_id import orient
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from sample.pipeline.functions.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from sample.pipeline.functions.experiment_class_surf import Experiment
from sample.paths.directory import run_parallel, find_state, directory_scratch, directory_project, path_code

from sample.notebooks.analysis.data_info import *
import matplotlib.patches as mpatches
from statsmodels.stats import weightstats as stests
window=800
results={}
for treatment in treatments.keys():
    insts = treatments[treatment]
    for inst in insts:
        results[inst] = pickle.load(open(f'{path_code}/MscThesis/Results/straight_{window}_{inst}.pick', "rb"))
column_names = ["plate","inst", "treatment", "angle", "curvature","density","growth","speed","straightness","t","hyph","total_network"]
infos = pd.DataFrame(columns=column_names)
for treatment in treatments.keys():
    insts = treatments[treatment]
    for inst in insts:
        angles, curvatures, densities,growths,speeds,tortuosities,ts,hyphs,total_lengths = results[inst]
        for i,angle in enumerate(angles):
            new_line = pd.DataFrame(
                {   "plate": [plate_number[inst]],
                    "inst": [inst],
                    "treatment": [treatment],
                    "angle": [angle],
                    "curvature": [curvatures[i]],
                    "density": [densities[i]],
                    "growth": [growths[i]],
                    "speed": [speeds[i]],
                    "straightness": [tortuosities[i]],
                     "t": [ts[i]],
                     "hyph": [hyphs[i]],
                     "total_network" : [total_lengths[i]],
                }
            )  # index 0 for
            # mothers need to be modified to resolve multi mother issue
            infos = infos.append(new_line, ignore_index=True)
corrected = infos.loc[(infos["straightness"] <= 1)&(infos["plate"] !=435)& (infos["speed"] >=25)& (infos["speed"] <400)]
plate_94 = corrected.loc[corrected ['plate']==94]


In [2]:
plate_94 = corrected.loc[(corrected ['plate']==94) & (corrected ['t']<=70)& (corrected ['speed']>=50)]
# plate_94 = corrected.loc[corrected ['plate']==94]


In [5]:
dummy_baits = [298,319,373,344]
P_N_crossing_P = [420,433,405,425,384]
P_N_crossing_N = [423,436,409,400,403]
dummy_baits_se = corrected.loc[corrected ['plate'].isin(dummy_baits)]
P_N_crossing_P_se = corrected.loc[corrected ['plate'].isin(P_N_crossing_P)]
P_N_crossing_N_se = corrected.loc[corrected ['plate'].isin(P_N_crossing_N)]
no_bait_se = corrected.loc[corrected ['treatment']=='25*']


In [27]:
fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
abcisse = 't'
plate_94 = dummy_baits_se
baits_sort = plate_94.sort_values("t")
N=160
moving_av = baits_sort.rolling(N).mean()
moving_std = baits_sort.rolling(N).std()


ax.set_xlabel('time(t)')
ax.set_ylabel('speed($\mu m. h^{-1}$)')
xlim = 40
# ax.set_xlim(0,30)
ax.set_ylim(0,400)
# slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))
x= range(-xlim,xlim)

# ax.scatter(plate_94[abcisse],plate_94['speed'],color='green',label='observed speeds')
line1 = ax.plot(moving_av[abcisse],moving_av['speed'],color='green',label = 'dummy baits',)
ax.plot(moving_av[abcisse],(moving_av['speed']+moving_std['speed']/np.sqrt(N)),color='red')
ax.plot(moving_av[abcisse],(moving_av['speed']-moving_std['speed']/np.sqrt(N)),color='red')
plate_94 = P_N_crossing_P_se
baits_sort = plate_94.sort_values("t")
N=160
moving_av = baits_sort.rolling(N).mean()
moving_std = baits_sort.rolling(N).std()


ax.set_xlabel('time(t)')
ax.set_ylabel('speed($\mu m. h^{-1}$)')
xlim = 40
# ax.set_xlim(0,30)
ax.set_ylim(0,400)
# slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))
x= range(-xlim,xlim)

# ax.scatter(plate_94[abcisse],plate_94['speed'],color='black',label='observed speeds')
line2 =ax.plot(moving_av[abcisse],moving_av['speed'],color='black',label = 'P_N_crossing_P',)
ax.plot(moving_av[abcisse],(moving_av['speed']+moving_std['speed']/np.sqrt(N)),color='red')
ax.plot(moving_av[abcisse],(moving_av['speed']-moving_std['speed']/np.sqrt(N)),color='red')
plate_94 = P_N_crossing_N_se
baits_sort = plate_94.sort_values("t")
N=160
moving_av = baits_sort.rolling(N).mean()
moving_std = baits_sort.rolling(N).std()


ax.set_xlabel('time(t)')
ax.set_ylabel('speed($\mu m. h^{-1}$)')
xlim = 40
# ax.set_xlim(0,30)
ax.set_ylim(0,400)
# slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))
x= range(-xlim,xlim)

# ax.scatter(plate_94[abcisse],plate_94['speed'],color='pink',label='observed speeds')
line3 =ax.plot(moving_av[abcisse],moving_av['speed'],color='pink',label = 'P_N_crossing_N',)
ax.plot(moving_av[abcisse],(moving_av['speed']+moving_std['speed']/np.sqrt(N)),color='red')
ax.plot(moving_av[abcisse],(moving_av['speed']-moving_std['speed']/np.sqrt(N)),color='red')
plate_94 = no_bait_se
baits_sort = plate_94.sort_values("t")
N=320
moving_av = baits_sort.rolling(N).mean()
moving_std = baits_sort.rolling(N).std()


ax.set_xlabel('time(t)')
ax.set_ylabel('speed($\mu m. h^{-1}$)')
xlim = 40
# ax.set_xlim(0,30)
ax.set_ylim(0,400)
# slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))
x= range(-xlim,xlim)

# ax.scatter(plate_94[abcisse],plate_94['speed'],color='blue',label='observed speeds')
line4 =ax.plot(moving_av[abcisse],moving_av['speed'],color='blue',label = 'no_baits',)
ax.plot(moving_av[abcisse],(moving_av['speed']+moving_std['speed']/np.sqrt(N)),color='red')
ax.plot(moving_av[abcisse],(moving_av['speed']-moving_std['speed']/np.sqrt(N)),color='red')

plt.legend((line1,line2,line3,line4),('dummy baits','P and N crossing P','P and N crossing N','no baits'),loc='upper center')

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [3]:
baits_sort = plate_94.sort_values("total_network")
N=80
moving_av = baits_sort.rolling(N).mean()
moving_std = baits_sort.rolling(N).std()

fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel('total network($\mu m$)')
ax.set_ylabel('speed($\mu m. h^{-1}$)')
xlim = 40
# ax.set_xlim(0,60)
ax.set_ylim(0,400)
# slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))
x= range(-xlim,xlim)

ax.scatter(plate_94["total_network"],plate_94['speed'],color='yellow',label='observed speeds')
ax.plot(moving_av["total_network"],moving_av['speed'],color='green',label = 'moving average')
ax.plot(moving_av["total_network"],(moving_av['speed']+moving_std['speed']/np.sqrt(N)),color='red',label = 'moving average')
ax.plot(moving_av["total_network"],(moving_av['speed']-moving_std['speed']/np.sqrt(N)),color='red',label = 'moving average')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
160**2*np.log(2)**2/(2*1000**2)*24**2,56/20*np.log(2)/1000*24**2

(3.542283981016115, 1.1179077728070796)

In [11]:
160**2*np.log(2)**2/(2*1000**2)*24**2 + 3*np.log(2)/1000*24**2

4.7400423090237

In [39]:
np.sqrt(4*np.log(2)/1500*24**2)

1.0318304460230256

In [40]:
30000/3600

8.333333333333334

In [5]:
0.80*24

19.200000000000003